In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import math
from cmdstanpy import CmdStanModel

In [2]:
# Prepare for the data
# 20,100,1000,5000
ibd = pd.read_json('ibd_admix_data.json')
ibd['index'] = ibd['index'].apply(tuple)
ibd['group'] = ibd['group'].apply(tuple)
grouped = ibd.groupby(['group','u','v'])
n_boot = 1000

columns = ['group','mean_fraction','mean_number','boot_var','p_0','u','v']
boot_data = pd.DataFrame(columns = columns)
for name, group in grouped:
    gg = name[0]
    u = name[1]
    v = name[2]
    non_zero = [i for i in group['fraction'] if i > 0]
    p_0 = 1- (len(non_zero)/len(group['fraction']))
    number_mean = group['number'].mean()
    sample_mean = group['fraction'].mean()
    bootstrap_means = []

    for _ in range(n_boot):
        sample = np.random.choice(group['fraction'], size=len(group['fraction']), replace=True)
        bootstrap_means.append(np.mean(sample))

# Estimate variance
    variance_estimate = np.var(bootstrap_means)
    new_row = [gg,sample_mean,number_mean,variance_estimate,p_0,u,v]
    boot_data.loc[len(boot_data)] = new_row

In [3]:
print(boot_data.head())

    group  mean_fraction   mean_number      boot_var       p_0     u     v
0  (0, 0)       0.980951  51662.005263  7.641545e-07  0.000000  0.00  0.35
1  (0, 0)       0.001801      0.963158  1.804066e-08  0.384211  0.35  0.40
2  (0, 0)       0.001754      0.831579  1.858368e-08  0.442105  0.40  0.45
3  (0, 0)       0.001190      0.505263  1.527299e-08  0.615789  0.45  0.50
4  (0, 0)       0.001057      0.405263  1.362892e-08  0.663158  0.50  0.55


In [4]:
boot_data['mean_fraction'] = boot_data['mean_fraction'] * 1000
boot_data['boot_var'] = boot_data['boot_var'] * 1000000
boot_data['group'] = boot_data['group'].apply(list)
json_ibd = boot_data.to_dict(orient='list')
print(json_ibd)
json_ibd['N_obs'] = len(boot_data)
json_ibd['length'] = 200
json_ibd['outgroup_N'] = 20000
json_ibd['ancestral_T'] = 1000
json_ibd['T2'] = 100
json_ibd['ancestral_N'] = 20000


{'group': [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 2], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [0, 3], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 1], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 3], [1, 3], [1, 3], [1, 3], [1, 3], [1, 3], [1, 3], [1, 3], [1, 3], [1, 3], [1, 3], [1, 3], [1, 3], [1, 3], [1, 3], [1, 3

In [10]:
print(boot_data.head())

    group  mean_fraction   mean_number  boot_var       p_0     u     v
0  [0, 0]     980.950671  51662.005263  0.764155  0.000000  0.00  0.35
1  [0, 0]       1.800605      0.963158  0.018041  0.384211  0.35  0.40
2  [0, 0]       1.754332      0.831579  0.018584  0.442105  0.40  0.45
3  [0, 0]       1.189532      0.505263  0.015273  0.615789  0.45  0.50
4  [0, 0]       1.057111      0.405263  0.013629  0.663158  0.50  0.55


In [8]:
model = CmdStanModel(stan_file="ture_ibd_popn4.stan")

12:41:23 - cmdstanpy - INFO - compiling stan file E:\admix_stan\inference\ture_ibd_popn4.stan to exe file E:\admix_stan\inference\ture_ibd_popn4.exe
12:41:36 - cmdstanpy - INFO - compiled model executable: E:\admix_stan\inference\ture_ibd_popn4.exe


In [9]:
model.variational(json_ibd)

12:41:40 - cmdstanpy - INFO - Chain [1] start processing
12:41:40 - cmdstanpy - INFO - Chain [1] done processing
12:41:40 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted


RuntimeError: Error during variational inference: Exception: normal_lpdf: Scale parameter is 0, but must be positive! (in 'ture_ibd_popn4.stan', line 69, column 8 to column 70)
	Exception: normal_lpdf: Scale parameter is 0, but must be positive! (in 'ture_ibd_popn4.stan', line 69, column 8 to column 70)
	Exception: normal_lpdf: Scale parameter is 0, but must be positive! (in 'ture_ibd_popn4.stan', line 69, column 8 to column 70)
	Exception: normal_lpdf: Scale parameter is 0, but must be positive! (in 'ture_ibd_popn4.stan', line 69, column 8 to column 70)
	Exception: normal_lpdf: Scale parameter is 0, but must be positive! (in 'ture_ibd_popn4.stan', line 69, column 8 to column 70)
	Exception: normal_lpdf: Scale parameter is 0, but must be positive! (in 'ture_ibd_popn4.stan', line 69, column 8 to column 70)
	Exception: normal_lpdf: Scale parameter is 0, but must be positive! (in 'ture_ibd_popn4.stan', line 69, column 8 to column 70)
	Exception: normal_lpdf: Scale parameter is 0, but must